In [11]:
import os
import re
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.geometry import mapping
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from scipy import ndimage
from functools import partial

In [12]:

# =========================================================
# 2) TRÍCH DATETIME TỪ FILENAME
# =========================================================
def extract_datetime_from_filename(path):
    filename = os.path.basename(path)

    # Kiểu 1: CAPE_20190401000000.tif
    m14 = re.search(r"(\d{14})", filename)
    if m14:
        return pd.to_datetime(m14.group(1), format="%Y%m%d%H%M%S", errors="coerce")

    # Kiểu 2: B04B_20190401.Z0000_TB.tif
    m_date = re.search(r"(\d{8})", filename)
    m_z = re.search(r"Z(\d{4})", filename)

    if m_date:
        date = m_date.group(1)
        if m_z:
            return pd.to_datetime(date + m_z.group(1), format="%Y%m%d%H%M", errors="coerce")
        return pd.to_datetime(date, format="%Y%m%d", errors="coerce")

    return pd.NaT


# =========================================================
# 3) LIST FILE
# =========================================================
def list_all_files(root):
    out = []
    for dp, _, files in os.walk(root):
        for f in files:
            if f.endswith(".tif") or f.endswith(".TIF"):
                out.append(os.path.join(dp, f))
    return out

In [13]:
# =========================================================
# 1) LOAD SHAPEFILE HÀ TĨNH
# =========================================================
shp_path = "../gadm41_VNM_shp"
vnm_gdf = gpd.read_file(shp_path)

# Lọc Hà Tĩnh
ht_gdf = vnm_gdf[vnm_gdf['VARNAME_1'] == 'Ha Tinh']

ht_union = ht_gdf.geometry.union_all()
ht_crs = ht_gdf.crs

In [14]:
def fast_fill_nodata_nearest(arr):
    """  gán cứng các giá trị NaN/Inf thành -9999.
    """
    mask = np.isnan(arr) | np.isinf(arr)
    tong = 0

    if mask.any():
        arr[mask] = -9999.0
        tong+=1

    # print("tong so nodata = ",tong)
    return arr

In [19]:
def analyze_hatinh_pixels(raster_path, gdf_boundary):
    """
    Đếm tổng số pixel thuộc ranh giới và số lượng pixel bị null/nodata.

    Args:
        raster_path (str): Đường dẫn đến file .tif
        gdf_boundary (GeoDataFrame): GeoDataFrame chứa ranh giới (ví dụ: ht_gdf)

    Returns:
        dict: Chứa thông tin thống kê (total_pixels, null_pixels, valid_pixels, null_ratio)
    """
    try:
        with rasterio.open(raster_path) as src:
            # 1. Đọc dữ liệu và chuyển sang float để xử lý NaN dễ hơn (giống code gốc của bạn)
            data = src.read(1).astype(float)
            nodata_val = src.nodata
            transform = src.transform
            src_crs = src.crs

            # 2. Xử lý CRS: Đảm bảo ranh giới khớp với hệ toạ độ của Raster
            # (Logic lấy từ bước 5 của code gốc)
            if src_crs != gdf_boundary.crs:
                geom = gdf_boundary.to_crs(src_crs).geometry.union_all()
            else:
                geom = gdf_boundary.geometry.union_all()

            # 3. Tạo Mask: Chỉ lấy pixel nằm TRONG ranh giới Hà Tĩnh
            # invert=True nghĩa là pixel nằm trong shape sẽ có giá trị True
            mask = geometry_mask(
                [mapping(geom)],
                invert=True,
                out_shape=data.shape,
                transform=transform
            )

            # 4. Trích xuất giá trị pixel thuộc Hà Tĩnh
            # Dùng boolean indexing để lấy mảng 1 chiều các giá trị bên trong mask
            ht_values = data[mask]

            total_pixels = ht_values.size

            if total_pixels == 0:
                print(f"⚠️ {os.path.basename(raster_path)}: Không có pixel nào thuộc Hà Tĩnh.")
                return None

            # 5. Đếm Null/Nodata
            # Pixel được coi là Null nếu:
            # - Bằng giá trị nodata_val (nếu có)
            # - Là np.nan (do data đã convert sang float)
            # - Là np.inf (vô cực)

            is_nan = np.isnan(ht_values) | np.isinf(ht_values)

            if nodata_val is not None:
                # Cẩn thận so sánh float: dùng np.isclose hoặc so sánh trực tiếp nếu chắc chắn
                is_nodata = (ht_values == nodata_val)
                null_mask = is_nan | is_nodata
            else:
                null_mask = is_nan

            null_count = np.sum(null_mask)
            valid_count = total_pixels - null_count
            null_ratio = (null_count / total_pixels) * 100

            if null_count == 0:
                return "kaka"

            # 6. In kết quả nhanh để kiểm tra
            print(f"--- File: {os.path.basename(raster_path)} ---")
            print(f"  > Tổng pixel Hà Tĩnh: {total_pixels:,}")
            print(f"  > Pixel Valid (có data): {valid_count:,}")
            print(f"  > Pixel Null/NoData:     {null_count:,} ({null_ratio:.2f}%)")

            return {
                "filename": os.path.basename(raster_path),
                "total_pixels": total_pixels,
                "null_pixels": null_count,
                "valid_pixels": valid_count,
                "null_ratio": null_ratio
            }

    except Exception as e:
        print(f"ERROR tại {os.path.basename(raster_path)}: {e}")
        return None

In [22]:
def test_ht():
    # Đảm bảo bạn đã load ht_gdf như code gốc
    # shp_path = "gadm41_VNM_shp"
    # vnm_gdf = gpd.read_file(shp_path)
    # ht_gdf = vnm_gdf[vnm_gdf['VARNAME_1'] == 'Ha Tinh']

    root = "DATA_SV/Precipitation/Radar" # Hoặc folder khác
    files = list_all_files(root)

    k= len(files)
    if len(files) > 0:
        # Test file đầu tiên tìm thấy
        count_not_null = 0
        for test_file in files:
            stats = analyze_hatinh_pixels(test_file, ht_gdf)
            if stats == "kaka":
                count_not_null += 1

        if count_not_null == k:
             print("ko thieu pixel nao ca")
test_ht()

ko thieu pixel nao ca


In [62]:
def extract_HaTinh_bbox_all_pixels(path, root):
    try:
        with rasterio.open(path) as src:
            data = src.read(1).astype(float)
            nodata = src.nodata
            transform = src.transform
            src_crs = src.crs
            H, W = data.shape

        # ---------------------------------------
        # 0) Chuẩn hoá dữ liệu
        # ---------------------------------------
        data[data == nodata] = np.nan
        data[data == -9999] = np.nan
        data[~np.isfinite(data)] = np.nan

        # ---------------------------------------
        # 1) Reproject lại shapefile
        # ---------------------------------------
        if src_crs != ht_crs:
            geom = ht_gdf.to_crs(src_crs).geometry.union_all()
        else:
            geom = ht_union

        # ---------------------------------------
        # 2) Tạo MASK cho Hà Tĩnh
        # ---------------------------------------
        mask_full = geometry_mask(
            [mapping(geom)],
            invert=True,
            out_shape=(H, W),
            transform=transform,
            all_touched=True
        )

        rows_full, cols_full = np.where(mask_full)

        if len(rows_full) == 0:
            print("WARNING: không có pixel nào trong Hà Tĩnh")
            return pd.DataFrame()

        # ---------------------------------------
        # 3) TÍNH BBOX CỐ ĐỊNH 21x34 TỪ TÂM HÀ TĨNH
        # ---------------------------------------

        # Tìm tâm của Hà Tĩnh (dựa trên mask)
        center_row = int(np.mean(rows_full))
        center_col = int(np.mean(cols_full))

        # Kích thước cố định
        n_rows_fixed = 21
        n_cols_fixed = 34

        # Tính min/max từ tâm
        half_rows = n_rows_fixed // 2
        half_cols = n_cols_fixed // 2

        min_row = center_row - half_rows
        max_row = center_row + half_rows

        # Kiểm tra số lẻ cho rows
        if n_rows_fixed % 2 == 0:
            max_row -= 1

        min_col = center_col - half_cols
        max_col = center_col + half_cols

        # Kiểm tra số lẻ cho cols
        if n_cols_fixed % 2 == 0:
            max_col -= 1

        # Đảm bảo không vượt quá biên ảnh
        min_row = max(0, min_row)
        min_col = max(0, min_col)
        max_row = min(H - 1, max_row)
        max_col = min(W - 1, max_col)

        # Tạo grid cho bounding box
        rows = np.arange(min_row, max_row + 1)
        cols = np.arange(min_col, max_col + 1)

        # Đảm bảo kích thước chính xác
        if len(rows) != n_rows_fixed:
            if len(rows) > n_rows_fixed:
                rows = rows[:n_rows_fixed]
            else:
                rows = np.arange(min_row, min_row + n_rows_fixed)

        if len(cols) != n_cols_fixed:
            if len(cols) > n_cols_fixed:
                cols = cols[:n_cols_fixed]
            else:
                cols = np.arange(min_col, min_col + n_cols_fixed)

        rgrid, cgrid = np.meshgrid(rows, cols, indexing='ij')
        rows_flat = rgrid.flatten()
        cols_flat = cgrid.flatten()

        # ---------------------------------------
        # 4) Giá trị pixel - SỬA THỨ TỰ XỬ LÝ
        # ---------------------------------------
        vals = data[rows_flat, cols_flat].copy()  # Dùng copy để tránh tham chiếu

        # Tạo mask cho bounding box
        mask_subset = mask_full[rows_flat, cols_flat]

        # DEBUG: Kiểm tra trạng thái ban đầu
        nan_mask_original = np.isnan(vals)
        print(f"DEBUG {os.path.basename(path)}: NaN ban đầu = {nan_mask_original.sum()}")

        # XỬ LÝ THEO THỨ TỰ ĐÚNG:
        # 1. Đầu tiên, xử lý tất cả NaN TRONG Hà Tĩnh → -9999
        in_ht_and_nan = mask_subset & nan_mask_original
        vals[in_ht_and_nan] = -9999

        # 2. Sau đó, xử lý tất cả NaN NGOÀI Hà Tĩnh → -1
        out_ht_and_nan = (~mask_subset) & nan_mask_original
        vals[out_ht_and_nan] = -1

        # 3. Cuối cùng, xử lý tất cả pixel NGOÀI Hà Tĩnh (không phải NaN) → -1
        out_ht_not_nan = (~mask_subset) & (~nan_mask_original)
        vals[out_ht_not_nan] = -1

        # ---------------------------------------
        # 5) DEBUG: Kiểm tra kết quả
        # ---------------------------------------
        n_total = len(vals)
        n_in_ht = mask_subset.sum()
        n_out_ht = len(vals) - n_in_ht
        n_9999 = (vals == -9999).sum()
        n_1 = (vals == -1).sum()
        n_valid = n_total - n_9999 - n_1

        print(f"  Tổng pixel: {n_total}")
        print(f"  Trong Hà Tĩnh: {n_in_ht}")
        print(f"  Ngoài Hà Tĩnh: {n_out_ht}")
        print(f"  Giá trị -9999: {n_9999}")
        print(f"  Giá trị -1: {n_1}")
        print(f"  Giá trị hợp lệ: {n_valid}")

        # ---------------------------------------
        # 6) Xuất CSV
        # ---------------------------------------
        lons, lats = rasterio.transform.xy(transform, rows_flat, cols_flat, offset='center')
        ts = extract_datetime_from_filename(path)
        rel = os.path.relpath(path, root)
        var = rel.split(os.sep)[0]

        return pd.DataFrame({
            "variable": var,
            "timestamp": ts,
            "row": rows_flat,
            "col": cols_flat,
            "lon": lons,
            "lat": lats,
            "value": vals,
            "in_ha_tinh": mask_subset.astype(int)
        })

    except Exception as e:
        print("ERROR:", path, e)
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

In [64]:

# =========================================================
# 6) MAIN
# =========================================================
def tif2csv():

    out_csv = "../csv_data/RADAR_hatinh_9999.csv"
    root = "../DATA_SV/Precipitation/Radar"

    # out_csv = "csv_data/HIMA_hatinh_rec.csv"
    # root = "DATA_SV/Hima"

    # out_csv = "csv_data/ERA5_hatinh_rec.csv"
    # root = "DATA_SV/ERA5"

    os.makedirs(os.path.dirname(out_csv), exist_ok=True)

    files = list_all_files(root)
    print("Tổng file tìm thấy:", len(files))

    if os.path.exists(out_csv):
        os.remove(out_csv)

    func = partial(extract_HaTinh_bbox_all_pixels, root=root)
    results = []

    with ThreadPoolExecutor(max_workers=10) as pool:
        futures = [pool.submit(func, f) for f in files]

        for f in tqdm(as_completed(futures), total=len(futures), desc="Process"):
            try:
                df = f.result()
                if df is not None and not df.empty:
                    results.append(df)
            except Exception as e:
                print("Thread error:", e)

    if results:
        final = pd.concat(results, ignore_index=True)
        final.to_csv(out_csv, index=False)
        print("DONE! Tổng pixel =", len(final))
    else:
        print("Không có data.")

tif2csv()

Tổng file tìm thấy: 2487


Process:   1%|          | 21/2487 [00:00<00:11, 205.55it/s]

DEBUG Radar_20190401000000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20190401010000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20190401030000.tif: NaN ban đầu = 0
DEBUG Radar_20190401020000.tif: NaN ban đầu = 0
DEBUG Radar_20190401040000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20190401070000.tif: NaN ban đầu = 0
DEBUG Radar_20190401060000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị

Process:   3%|▎         | 63/2487 [00:00<00:12, 200.20it/s]

DEBUG Radar_20190402180000.tif: NaN ban đầu = 0
DEBUG Radar_20190402190000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20190402200000.tif: NaN ban đầu = 0
DEBUG Radar_20190402230000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20190402220000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20190403000000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 37

...

DEBUG Radar_20201029060000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201029070000.tif: NaN ban đầu = 0
DEBUG Radar_20201029080000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201029090000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201029100000.tif: NaN ban đầu = 0


Process:  98%|█████████▊| 2446/2487 [00:12<00:00, 203.37it/s]

DEBUG Radar_20201029120000.tif: NaN ban đầu = 0  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378

DEBUG Radar_20201029110000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201029150000.tif: NaN ban đầu = 0
DEBUG Radar_20201029140000.tif: NaN ban đầu = 0
DEBUG Radar_20201029130000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201029160000.tif: NaN ban đầu = 

Process: 100%|██████████| 2487/2487 [00:12<00:00, 198.47it/s]


DEBUG Radar_20201031080000.tif: NaN ban đầu = 0DEBUG Radar_20201031070000.tif: NaN ban đầu = 0

  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201031090000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201031100000.tif: NaN ban đầu = 0
DEBUG Radar_20201031110000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị -9999: 0
  Giá trị -1: 336
  Giá trị hợp lệ: 378
DEBUG Radar_20201031150000.tif: NaN ban đầu = 0
DEBUG Radar_20201031140000.tif: NaN ban đầu = 0
  Tổng pixel: 714
  Trong Hà Tĩnh: 378
  Ngoài Hà Tĩnh: 336
  Giá trị

In [66]:
df_all1 = pd.read_csv('csv_data/RADAR_hatinh_9999.csv')

# Lấy một timestamp bất kỳ để kiểm tra
sample_df = df_all1[df_all1['timestamp'] == df_all1['timestamp'].iloc[0]]

min_row1, max_row1 = sample_df["row"].min(), sample_df["row"].max()
min_col1, max_col1 = sample_df["col"].min(), sample_df["col"].max()
n_row1 = int(max_row1 - min_row1 + 1)
n_col1 = int(max_col1 - min_col1 + 1)

print(f"Kích thước bounding box: {n_row1} x {n_col1}")
print(f"min_row: {min_row1}, max_row: {max_row1}")
print(f"min_col: {min_col1}, max_col: {max_col1}")

# Kiểm tra số lượng pixel unique
unique_rows = sample_df['row'].nunique()
unique_cols = sample_df['col'].nunique()
print(f"Số dòng unique: {unique_rows}")
print(f"Số cột unique: {unique_cols}")

Kích thước bounding box: 21 x 34
min_row: 59, max_row: 79
min_col: 101, max_col: 134
Số dòng unique: 21
Số cột unique: 34


In [61]:
print(sample_df)

     variable            timestamp  row  col     lon    lat  value  in_ha_tinh
0        2019  2019-04-01 00:00:00   59  101  105.06  18.72   -1.0           0
1        2019  2019-04-01 00:00:00   59  102  105.10  18.72   -1.0           0
2        2019  2019-04-01 00:00:00   59  103  105.14  18.72   -1.0           0
3        2019  2019-04-01 00:00:00   59  104  105.18  18.72   -1.0           0
4        2019  2019-04-01 00:00:00   59  105  105.22  18.72   -1.0           0
..        ...                  ...  ...  ...     ...    ...    ...         ...
709      2019  2019-04-01 00:00:00   79  130  106.22  17.92    0.0           1
710      2019  2019-04-01 00:00:00   79  131  106.26  17.92    0.0           1
711      2019  2019-04-01 00:00:00   79  132  106.30  17.92    0.0           1
712      2019  2019-04-01 00:00:00   79  133  106.34  17.92   -1.0           0
713      2019  2019-04-01 00:00:00   79  134  106.38  17.92   -1.0           0

[714 rows x 8 columns]
